# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [4]:
# imports
import os
import sys
import json
import requests
from IPython.display import display, Markdown, update_display

# Add the week1 directory to Python path to find scraper module
week1_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if week1_dir not in sys.path:
    sys.path.insert(0, week1_dir)

from openai import OpenAI
from dotenv import load_dotenv
from scraper import fetch_website_contents, fetch_website_links

In [5]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

# Set up OpenAI API client
# load API key from .env file
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# validate API key
if api_key and api_key.startswith("sk-proj-") and len(api_key) > 10:
    print("API key loaded successfully.")
else:
    raise ValueError("Invalid or missing OpenAI API key. Please set the OPENAI_API_KEY environment variable.")

# initialize OpenAI client
openai = OpenAI(api_key=api_key)

# Set up Ollama API client
# Ensure Ollama server is running
requests.get("http://localhost:11434").content 

# initialize Ollama client
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="")

API key loaded successfully.


In [6]:
# set up environment
systemPrompt = """
You are a helpful assistant that is expert in AI, LLMs, python programming, and data science. 
You help users by explaining concepts, code snippets, and best practices in a clear and concise manner.
"""




In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
def get_user_prompt(question=""):
    user_prompt = f"""
    Please answer the following question in detail and provide code examples where applicable:
    """
    if not question:
        question = input("Question cannot be empty. Enter your question: ")

    user_prompt += question
    return user_prompt


In [8]:
# Get gpt-4o-mini to answer, with streaming
def get_gpt_answer(question):
    stream = openai.chat.completions.create(
        model = MODEL_GPT,
        messages=[
            {"role": "system", "content": systemPrompt},
            {"role": "user", "content": get_user_prompt(question)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

get_gpt_answer(question)

The provided code snippet uses a generator expression along with the `yield from` statement in Python. Let's break down what it does step-by-step.

### Code Explanation

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. **Context of `yield from`:**
   - The `yield from` statement is used in generator functions to yield values from another generator or iterable. It simplifies the process of returning multiple values to the caller.

2. **Set Comprehension:**
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
   - It constructs a set of authors from a collection of `books`.
   - Here’s how it works:
     - `book.get("author")`: The `get` method attempts to retrieve the value associated with the key `"author"` from each `book` dictionary. If the key does not exist, it will return `None` (the default behavior of `get`).
     - The loop `for book in books` iterates over each element (book) in the `books` iterable.
     - The condition `if book.get("author")` filters out any books where the author is `None` (or the key does not exist), so only books with an actual author value will be included in the set.

3. **Setting it Together:**
   - The entire expression `{book.get("author") for book in books if book.get("author")}` results in a set of unique authors from the `books` collection, ensuring that each author only appears once in the result.

4. **Usage Considerations:**
   - After the set is created by the comprehension, `yield from` will yield each author in the set one by one to the caller of the generator function.

### Example

To better illustrate what this code does, let's consider an example:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4"},   # No author key
    {"title": "Book 5", "author": "Author A"},  # Duplicate author
]

for author in get_authors(books):
    print(author)
```

### Output
```
Author A
Author B
```

### Explanation of the Example

- In the example, we populate a list of dictionaries, where each dictionary represents a book with a title and an optional author.
- The `get_authors` function is a generator that uses the `yield from` statement with a set comprehension to yield unique authors of the provided list of books.
- When the generator is iterated over, it yields "Author A" and "Author B". "Author A" appears only once despite appearing multiple times in the `books` list, due to the properties of the set.

### Summary
The code snippet efficiently collects and yields unique authors from a collection of book dictionaries, ignoring any entries without valid authors, and encapsulates this logic within a generator function for convenient and deferred execution.

In [10]:
# Get Llama 3.2 to answer

def get_llama_answer(question):
    stream = ollama.chat.completions.create(
        model = MODEL_LLAMA,
        messages=[
            {"role": "system", "content": systemPrompt},
            {"role": "user", "content": get_user_prompt(question)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

get_llama_answer(question)

**Detailed Explanation**

The given code snippet is a part of Python 3.3+ syntax, which uses the `yield from` statement to delegate to a subiterable (in this case, another generator expression) and allows the execution of the surrounding function to be suspended while waiting for a value from the subiterable.

Here's a breakdown of what each part of the code does:

1. `{book.get("author") for book in books if book.get("author")}`: This is a generator expression that:
	* Iterates over a list or collection of dictionaries (`books`).
	* For each dictionary, it attempts to access the `"author"` key using `dict.get()`. If the key does not exist, an empty string (`''`) is used as its value.
	* Returns the author's name (string) if the key exists.
2. `yield from`: This keyword takes the generator expression above and delegates its execution to it. It means the surrounding function will:
	* Pause execution when a value is not available from the subiterable.
	* Resume execution with a value yielded by the subiterable.

**How it Works**

Imagine you have a large collection of book data, where each book has an `"author"` key. You want to yield all the authors' names, but only those who have actual values associated with the `"author"` key in their dictionary.

Here's how `yield from` simplifies this process:

1. Start executing the function.
2. Iterate over `books`: Look at each book in the collection.
3. For each book:
	* Check if `"author"` exists. If it doesn't, skip to the next iteration.
	* Yield the author's name using `yield from`.
4. Execute any remaining iterations.

**Code Simplification**

If you were to write this code without `yield from`, you might do something like:

```python
for book in books:
    for field in ["author"]:  # simplified iterator over fields
        if field in book and book[field]:
            yield (book, field)
```

Or, more efficiently:

```python
for book in books:
    author = book.get("author")
    if author:
        # yielding individual authors might be desired
        for _ in range(len(author)):  # using tuple unpacking for 'author'
            name = next(iter(author))
            yield name
```

However, `yield from` is not only cleaner but also more Pythonic and readable.

In your original snippet:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

* Iterating over the values of a dictionary.
* Filtering out items based on conditions (books with no `"author"` will be skipped).
* Yielding individual authors as a result.

Overall, `yield from` simplifies code while ensuring that it correctly handles iterables, reduces unnecessary iterations, and is an efficient way to execute multiple generator functions within another function.